In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam, SGD
import matplotlib.pyplot as plt


data = pd.read_csv('/content/AirPassengers.csv')
data = data[['#Passengers']]


scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

def create_sequences(data, sequence_length):
    x, y = [], []
    for i in range(len(data) - sequence_length):
        x.append(data[i:i + sequence_length])
        y.append(data[i + sequence_length])
    return np.array(x), np.array(y)

sequence_length = 50
X, y = create_sequences(scaled_data, sequence_length)
X = np.expand_dims(X, axis=-1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


def build_model(units, learning_rate, optimizer, activation, dropout_rate, num_layers):
    model = Sequential([Input(shape=(sequence_length, 1))])
    for i in range(num_layers - 1):
        model.add(LSTM(units, activation=activation, return_sequences=True))
        model.add(Dropout(dropout_rate))
    model.add(LSTM(units, activation=activation, return_sequences=False))
    model.compile(optimizer=optimizer(learning_rate=learning_rate), loss='mse')
    return model

result = []

def run_model(activation, optimizer, num_layers, units, epochs, batch_size):
    model = build_model(units, 0.001, optimizer, activation, 0.02, num_layers)
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(X_test, y_test), verbose=0)
    train_loss = history.history['loss'][-1]
    test_loss = history.history['val_loss'][-1]

    result.append({
        "Neurons": units,
        "Num Layers": num_layers,

        "Activation": activation,
        "Epochs": epochs,
        "Optimizer": optimizer.__name__,



        "Batch Size": batch_size,
        "Train Loss": train_loss,
        "Test Loss": test_loss
    })
    print(f"Activation: {activation}, Optimizer: {optimizer.__name__}, Num Layers: {num_layers}, Neurons: {units}, "
          f"Epochs: {epochs}, Batch Size: {batch_size}, Train Loss: {train_loss}, Test Loss: {test_loss}")

for i in range(8):
    activation = np.random.choice(['relu', 'tanh', 'sigmoid'])
    optimizer = np.random.choice([Adam, SGD])
    num_layers = int(np.random.choice([1, 2, 3]))
    units = int(np.random.choice([32, 64, 128, 256]))
    epochs = int(np.random.choice([20, 50, 100]))
    batch_size = int(np.random.choice([16, 32, 64]))

    run_model(activation, optimizer, num_layers, units, epochs, batch_size)


result_df = pd.DataFrame(result)
result_df.to_csv('/content/B190305017.csv', index=False)
print("Results saved to 'model_results.csv'")
